In [1]:
# 코드 사용에 필요한 라이브러리 불러오기
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

In [2]:
# parsing Arguments
# python DCGAN practice.py 
# python DCGAN practice.py -h  : help 
def parse_args():
    parser = argparse.ArgumentParser()
    
    # 11개의 parser
    parser.add_argument("--n_epochs", type=int, default=500, help="number of epochs for training")
    parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
    parser.add_argument("--lr", type=float, default=5e-4, help="adam: learning rate")
    parser.add_argument("--b1", type=float, default=0.5, help="decay of first order momentum of gradient")
    parser.add_argument("--b2", type=float, default=0.999, help="decay of first order momentum of gradient")
    # 잠재 공간
    parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
    parser.add_argument("--img_size", type=int, default=48, help="size of each image dimension")
    parser.add_argument("--channels", type=int, default=3, help="number of image channels")
    parser.add_argument("--sample_interval", type=int, default=400, help="interval between image samples")
    parser.add_argument("--seed", type=int, default=777, help="seed number")
    parser.add_argument("--object", type=str, default='data', help="which object to generate")
    
    args.parser.parse_args()
    
    return args

In [3]:
# dataloader
def set_dataloader(args):
    
    # args.object = "--object"
    dataset = datasets.ImageFolder(root='.data/{}'.format(args.object),
                                   transforms=transforms.Compose([
                                       transforms.Resize((args.img_size, args.img_size)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5], [0.5])
                                   ]))
    dataloader = DataLoader(dataset=dataset,
                           batch_size=args.batch_size,
                           shuffle=True)
    return dataloader

In [4]:
# Generator
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Generator(nn.Module):
    def __init__(self, latent_dim, image_shape):
        super(Generator, self).__init__()
        
        self.latent_dim = latent_dim 
        self.image_shape = image_shape
        
        def block(input_fea, output_fea, normalize=True): # 함수내에 함수 선언 형태
            layers = [nn.Linear(input_fea, output_fea)]
            if normalize:
                layers.append(nn.BatchNorm1d(output_fea, 0.5))

            layers.append(nn.LeakyReLU(0.2, inplace=True))

            return layers
        
        # 인자길이를 모를 떄 길이를 가변 가능하도록 * 붙임
        # * args : 하나의 argument
        # ** ketargs : keyword argument(딕셔너리 형태의 인자들)
        self.model = nn.Sequential(
        *block(latent_dim, 128, normalize=False),
        *block(128, 256),
        *block(512, 1024),
        nn.Linear(1024, int(np.prod(image_shape))),
        nn.Tanh()
        )
        
    def forward(self, z):
        image = self.model(z)
        image = image.view(image.size(0), *self.image_shape)
        
        return image

In [5]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, image_shape):
        super(Discriminator, self).__init__()
        
        # CNN 구조
        self.model = nn.Sequential(
        nn.Linear(int(np.prod(image_shape)), 512), 
                 nn.LeakyReLU(0.2, inplace=True),
            
                 nn.Linear(512, 256),
                 nn.LeakyReLU(0.2, inplace=True),
            
                 nn.Linear(256, 1),
                 nn.Sigmoid()
        )
        
    def forward(self, image):
        image_flat = image.view(image.size(0), -1)
        validity = self.model(image_flat)
        
        return validity

In [6]:
# seed 설정과 하이퍼 파라미터 선언
import random
def set_seed(args): # 값이 변화하는 것을 막아주기
    random.seed(args.seed) # paser의 args에 접근
    np.randomseed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    
def main(): # main을 통해 args를 접근
    # seed setting
    set_seed(args)
    dataloader = set_dataloader(args)
    
    image_shape = (args.channels, arg.img_size, args.img_size) # rgb(3), 48 * 48
    
    # 한줄로도 표현 가능한 코드
    if torch.cuda.is_available():
        device = torch.device("cuda : 0")
        Tensor = torch.cuda.FloatTensor
        
    else:
        device = torch.device("cpu")
        Tensor = torch.FloatTensor
    
    print("Current Device : {} \t Base Tensor : {}".format(device, Tensor))
    
    criterion = torch.nn.BCELoss().to(device)
    generator = Generator(args.latent_dim, image_shape).to(device)
    discriminator = Discriminator(image_shape).to(device)
    
    # betas : Adam에서 사용하는 momentum의 개수
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=args.lr, betas=(args.b1, args.b2))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=args.lr, betas=(args.b1, args.b2))

In [7]:
# training
import os
from torch.autograd import Variable
from torchvision.utils import save_image

from datetime import datetime

def train(args, device, dataloader, criterion, generator, discriminator, optimizer_G, optimizer_D, Tensor):
    experiment_time = datetime.today().strftime("%Y%m%d_%H_%M") # 얼마나 시간이 걸렸는지
    result_dir = 'images/{}'.format(experiment_time) # str 형식으로 폴더에 저장
    model_dir = 'trained_model/{}'.format(experiment_time)
    
    os.makedirs(result_dir, exist_ok=False) # 기존에 있는 폴더를 사용한다
    os.makedirs(model_dir, exist_ok=False)
    
    for epoch in range(args.n_epochs):
        for idx, data in enumerate(dataloader): # index
            images, labels = data[0].to(device), data[1].to(device)
            
            # 실제 이미지(Ground Truth)
            valid = Variable(Tensor(images.size(0), 1).fill_(1.0), requires_grad=False)
            fake = Variable(Tensor(images.size(0), 1).fill(0.0), requires_grad=False)
            
            real_images = Variable(image.type(Tensor))
            
            ## generator training
            optimizer_G.zero_grad()
            # optimizer.G.zero_grad()
            
            # generator sample noise input
            z = Variable(Tensor(np.random.normal(0, 1, (images.size(0), args.latent_dim))))
            
            # generator가 만들어낸 가짜 이미지
            gen_images = generator(z)
            
            loss_G = criterion(discriminator(gen_images), valid)
            
            loss_G.backward()
            optimizer_G.step()
            
            ## discriminator training
            optimizer_D.zero_grad()
            
            loss_real = criterion(discriminator(real_images), valid)
            loss_fake = criterion(discriminator(gen_images.detach()), fake)
            loss_D = (loss_real + loss_fake) / 2
            
            loss_D.backward()
            optimizer_D.step()
            
            # training ...
            if idx % 20 == 0:
                print("[Epoch: {:d}/{:d}] \t [Batch {:d}/{:d}] \t [Loss_G {:.4f}] \t [Loss_D {:.4f}]"
                     .format(epoch, args.n_epochs, idx, len(dataloader), loss_G.item(), loss_D.item()))
            
            batches_done = epoch * len(dataloader) + idx
            
            if batches_done % args.sample_interval == 0:
                print('Save sample image')
                # batch가 끝났을 때 이미지 저장
                save_image(gen_images.data[:25],'{}/{:d}.png'
                           .format(result_dir, batches_done), nrow=5, normalize=True)
                
        print('Everything Done... Saving Model')
        
        # save model
        PATH_G = model_dir + '/generator.pth'
        PATH_D = model_dir + '/discriminator.pth'
        
        # save
        torch.save(generator.state_dict(), PATH_G)
        torch.save(discriminator.state_dict(), PATH_D)